In [ ]:
import pandas as pd
import numpy as np
import os
import xgboost as xgb

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

In [2]:
# ------------------- Đọc và chuẩn bị dữ liệu -------------------
df = pd.read_csv("data_gia.csv")

# Tách đặc trưng và nhãn
X = df.drop("Giá bán (VNĐ)", axis=1)
y = df["Giá bán (VNĐ)"]

# Chia tập train và test (80% - 20%)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [3]:
# ------------------- Huấn luyện mô hình -------------------
model = xgb.XGBRegressor(
    objective='reg:squarederror', 
    n_estimators=100, 
    learning_rate=0.1, 
    max_depth=4, 
    random_state=42
)

model.fit(X_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.1, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=4, max_leaves=0, min_child_weight=1,
             missing=nan, monotone_constraints='()', n_estimators=100, n_jobs=0,
             num_parallel_tree=1, predictor='auto', random_state=42,
             reg_alpha=0, reg_lambda=1, ...)

In [4]:
# ------------------- Đánh giá mô hình -------------------
y_pred = model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)
mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100
accuracy = 100 - mape

print(f"MSE: {mse:,.2f}")
print(f"RMSE: {rmse:,.2f}")
print(f"R2 Score: {r2:.4f}")
print(f"Độ chính xác (MAPE-based): {accuracy:.2f}%")

MSE: 17,476,879,828,206,442.00
RMSE: 132,200,150.64
R2 Score: 0.3814
Độ chính xác (MAPE-based): 80.99%


In [5]:
# ------------------- Lưu mô hình -------------------
save_dir = "exported_model"
os.makedirs(save_dir, exist_ok=True)

model_path = os.path.join(save_dir, "xgboost_model.json")
model.save_model(model_path)

print(f"Mô hình đã được lưu vào file: '{model_path}'")

Mô hình đã được lưu vào file: 'exported_model\xgboost_model.json'


In [6]:
# ------------------- Dự đoán với dữ liệu mới -------------------
new_house = pd.DataFrame({
    "Diện tích (m2)": [120],
    "Số phòng ngủ": [3],
    "Số phòng tắm": [2],
    "Năm xây dựng": [1990],
    "Vị trí (Mã vùng)": [55156]
})

# Đảm bảo kiểu dữ liệu khớp
new_house = new_house.astype(X.dtypes.to_dict())

predicted_price = model.predict(new_house)[0]
print(f"\nGiá bán dự đoán cho căn nhà mới: {predicted_price:,.0f} VNĐ")


Giá bán dự đoán cho căn nhà mới: 598,004,480 VNĐ
